# 快速开始序贯（Sequential）模型

序贯模型是多个网络层的线性堆叠，也就是“一条路走到黑”。

可以通过向Sequential模型传递一个layer的list来构造该模型：

``` python
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(32, units=784),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])
```

也可以通过.add()方法一个个的将layer加入模型中：
``` python
model = Sequential()
model.add(Dense(32, input_shape=(784, )))
model.add(Activation('relu'))
```

## 指定输入数据的shape

模型需要知道输入数据的shape，因此，Sequential的第一层需要接受一个关于输入数据shape的参数，后面的各个层则可以自动的推导出中间数据的shape，因此不需要为每个层都指定这个参数。有几种方法来为第一层指定输入数据的shape：
- 传递一个input_shape的关键字参数给第一层，input_shape是一个tuple类型的数据，其中也可以填入None，如果填入None则表示此位置可能是任何正整数。数据的batch大小不应该包含在其中。
- 有些2D层，如Dense，支持通过指定其输入维度input_dim来隐含的指定输入数据shape，是一个Int类型的数据。一些3D的域层支持通过参数input_dim和input_length来指定输入shape。
- 如果你需要为输入指定一个固定大小的batch_size（常常用于stateful RNN网络），可以传递batch_size参数到一个层中，例如你想指定输入张量的batch的大小是32，数据shape是(6, 8)，则你需要传递batch_size=32和input_shape=(6, 8)

``` python
model = Sequential()
model.add(Dense(32, input_dim=784))
```

``` python
model = Sequential()
model.add(Dense(32, input_shape=(784, )))
```

## 编译

在训练模型之前，我们需要通过compile来对学习过程进行配置。compile接收三个参数：
- 优化器optimizer：该参数可指定为已预定义的优化器名，如rmsprop、adagrad，或一个Optimizer类的对象，详情见[optimizers](https://keras-cn.readthedocs.io/en/latest/other/optimizers/)。
- 损失函数loss：该参数为模型试图最小化的目标函数，它可为预定义的损失函数名，如categorical_crossentropy、mse，也可以为一个损失函数。详情见[losses](https://keras-cn.readthedocs.io/en/latest/other/objectives/)。
- 指标列表metrics：对分类问题，我们一般将该列表设置为metrics=\['accuracy'\]。指标可以是一个预定义指标的名字，也可以是一个用户定制的函数。指标函数应该返回单个张量，或一个完成metric_name->metric_value映射的字典。请参考[性能评估](https://keras-cn.readthedocs.io/en/latest/getting_started/other/metrices.md)。

``` python
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

# For custom metrics
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])
```

## 训练

Keras以Numpy数组作为输入数据和标签的数据类型。训练模型一般使用fit函数，该函数的详情见[这里](https://keras-cn.readthedocs.io/en/latest/models/sequential/)。下面是一些例子。
``` python
# For a single-input model with 2 classes (binary classification):
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)
```
<hr>
``` python
# For a single-input model with 10 classes (categorical classification):
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32)
```

## 例子

这里是一些帮助你开始的例子。

在Keras代码包的examples文件夹中，你将找到使用真实数据的示例模型：
- CIFAR10小图片分类：使用CNN和实时数据提升
- IMDB电影评论观点分类：使用LSTM处理成序列的词语
- Reuters(路透社)新闻主题分类：使用多层感知器（MLP）
- MNIST手写数字识别：使用多层感知器和CNN
- 字符级文本生成：使用LSTM

<font color=#0099ff size=4 face="黑体">基于多层感知器的softmax分类</font>

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
print(score)

Epoch 1/20
1000/1000 [==============================] - 0s 88us/step - loss: 2.4432 - acc: 0.0990
Epoch 2/20
1000/1000 [==============================] - 0s 16us/step - loss: 2.3545 - acc: 0.1100
Epoch 3/20
1000/1000 [==============================] - 0s 15us/step - loss: 2.3441 - acc: 0.0990
Epoch 4/20
1000/1000 [==============================] - 0s 15us/step - loss: 2.3280 - acc: 0.0980
Epoch 5/20
1000/1000 [==============================] - 0s 15us/step - loss: 2.3075 - acc: 0.1140
Epoch 6/20
1000/1000 [==============================] - 0s 15us/step - loss: 2.3148 - acc: 0.1050
Epoch 7/20
1000/1000 [==============================] - 0s 15us/step - loss: 2.3141 - acc: 0.1060
Epoch 8/20
1000/1000 [==============================] - 0s 15us/step - loss: 2.3165 - acc: 0.1110
Epoch 9/20
1000/1000 [==============================] - 0s 15us/step - loss: 2.3142 - acc: 0.1070
Epoch 10/20
1000/1000 [==============================] - 0s 16us/step - loss: 2.3130 - acc: 0.0930
Epoch 11/20
1000/10

<font color=#0099ff size=4 face="黑体">MLP的二分类</font>

In [3]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
print('test_loss:', score[0], '\ntest_accuracy:', score[1])

Epoch 1/20
1000/1000 [==============================] - 0s 158us/step - loss: 0.7504 - acc: 0.4820
Epoch 2/20
1000/1000 [==============================] - 0s 13us/step - loss: 0.7154 - acc: 0.5010
Epoch 3/20
1000/1000 [==============================] - 0s 14us/step - loss: 0.7191 - acc: 0.4870
Epoch 4/20
1000/1000 [==============================] - 0s 13us/step - loss: 0.7083 - acc: 0.5090
Epoch 5/20
1000/1000 [==============================] - 0s 14us/step - loss: 0.7076 - acc: 0.4830
Epoch 6/20
1000/1000 [==============================] - 0s 13us/step - loss: 0.7078 - acc: 0.4860
Epoch 7/20
1000/1000 [==============================] - 0s 15us/step - loss: 0.7035 - acc: 0.4970
Epoch 8/20
1000/1000 [==============================] - 0s 14us/step - loss: 0.6991 - acc: 0.5320
Epoch 9/20
1000/1000 [==============================] - 0s 14us/step - loss: 0.6968 - acc: 0.5100
Epoch 10/20
1000/1000 [==============================] - 0s 13us/step - loss: 0.6933 - acc: 0.5190
Epoch 11/20
1000/1

<font color=#0099ff size=4 face="黑体">类似VGG的卷积神经网络</font>

In [4]:
from __future__ import print_function

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-06, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)
print(score)

Epoch 1/10
100/100 [==============================] - 3s 34ms/step - loss: 2.3092
Epoch 2/10
100/100 [==============================] - 3s 31ms/step - loss: 2.2954
Epoch 3/10
100/100 [==============================] - 3s 31ms/step - loss: 2.2777
Epoch 4/10
100/100 [==============================] - 3s 31ms/step - loss: 2.3074
Epoch 5/10
100/100 [==============================] - 3s 31ms/step - loss: 2.2773
Epoch 6/10
100/100 [==============================] - 3s 32ms/step - loss: 2.2819
Epoch 7/10
100/100 [==============================] - 3s 33ms/step - loss: 2.2649
Epoch 8/10
100/100 [==============================] - 3s 32ms/step - loss: 2.2653
Epoch 9/10
100/100 [==============================] - 3s 31ms/step - loss: 2.2873
Epoch 10/10
20/20 [==============================] - 0s 12ms/step
2.29371023178


<font color=#0099ff size=4 face="黑体">使用LSTM的序列分类</font>

In [5]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
maxlen = 80 # cut texts after this number of words (among top max_features most common words)
batch_size=32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x times)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train, batch_size=batch_size, epochs=15, 
          validation_data=(x_test, y_test))

score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
17473536/17464789 [==============================] - 834s 48us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x times)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 85s 3ms/step - loss: 0.4596 - acc: 0.7820 - val_loss: 0.3853 - val_acc: 0.8338
Epoch 2/15
25000/25000 [==============================] - 86s 3ms/step - loss: 0.2955 - acc: 0.8801 - val_loss: 0.3709 - val_acc: 0.8372
Epoch 3/15
25000/25000 [==============================] - 86s 3ms/step - loss: 0.2197 - acc: 0.9138 - val_loss: 0.4337 - val_acc: 0.8244
Epoch 4/15
25000/25000 [==============================] - 86s 3ms/step - loss: 0.1523 - acc: 0.9421 - val_loss: 0.4896 - val_acc: 0.8307
Epoch 5/15
25000/25000 [==============================] - 86s 3ms/step - loss: 0.1083 - acc: 0.9608 - val_loss: 0.5839 - val_acc: 0.8221
Epoch 6/15
25000/25000 [=====

<font color=#0099ff size=4 face="黑体">使用1D卷积的序列分类</font>

In [8]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 15

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters, kernel_size, padding='valid', 
                 activation='relu', strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# we add a vanilla hidden layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# we project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, 
          epochs=epochs, validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 114s 5ms/step - loss: 0.4056 - acc: 0.7994 - val_loss: 0.3166 - val_acc: 0.8618
Epoch 2/15
25000/25000 [==============================] - 114s 5ms/step - loss: 0.2287 - acc: 0.9076 - val_loss: 0.2922 - val_acc: 0.8774
Epoch 3/15
25000/25000 [==============================] - 115s 5ms/step - loss: 0.1668 - acc: 0.9373 - val_loss: 0.2692 - val_acc: 0.8937
Epoch 4/15
25000/25000 [==============================] - 115s 5ms/step - loss: 0.1142 - acc: 0.9580 - val_loss: 0.3061 - val_acc: 0.8905
Epoch 5/15
25000/25000 [==============================] - 115s 5ms/step - loss: 0.0761 - acc: 0.9724 - val_loss: 0.3577 - val_acc: 0.8866
Epoch 6/15
25000/25000 [==============================] - 115s 5ms/step - loss: 0.0567 - acc: 0.9794

<font color=#0099ff size=4 face="黑体">用于序列分类的栈式LSTM</font>

在该模型中，我们将三个LSTM堆叠在一起，使该模型能够学习更高层次的时域特征表示。

开始的两层LSTM返回其全部输出序列，而第三层LSTM只返回其输出序列的最后一步结果，从而其时域维度降低（即将输入序列转换为单个向量）。

![栈式LSTM](./5.attaches/regular_stacked_lstm.png)

In [16]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
# returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True, input_shape=(timesteps, data_dim)))
# returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))
# returns a single vector of dimension 32
model.add(LSTM(32))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

score = model.evaluate(x_val, y_val, batch_size=32)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 2s 2ms/step - loss: 11.6169 - acc: 0.1020 - val_loss: 11.5961 - val_acc: 0.0700
Epoch 2/5
1000/1000 [==============================] - 0s 276us/step - loss: 11.6144 - acc: 0.1060 - val_loss: 11.5923 - val_acc: 0.1000
Epoch 3/5
1000/1000 [==============================] - 0s 284us/step - loss: 11.6144 - acc: 0.0960 - val_loss: 11.5923 - val_acc: 0.1200
Epoch 4/5
1000/1000 [==============================] - 0s 275us/step - loss: 11.6143 - acc: 0.1010 - val_loss: 11.5929 - val_acc: 0.0700
Epoch 5/5
100/100 [==============================] - 0s 126us/step
Test loss: 11.5940558624
Test accuracy: 0.11


<font color=#0099ff size=4 face="黑体">采用stateful LSTM的相同模型</font>

stateful LSTM的特点是，在处理过一个batch的训练数据后，其内部状态（记忆）会被作为下一个batch的训练数据的初始状态。状态LSTM使得我们可以在合理的计算复杂度内处理较长序列。

请在FAQ中关于[stateful LSTM](https://keras-cn.readthedocs.io/en/latest/for_beginners/FAQ/)的部分获取更多信息。

In [17]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10
batch_size = 32

# Expected input batch shape: (batch_size, timesteps, data_dim)
# Note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.
model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True, 
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(32, return_sequences=True, stateful=True))
model.add(LSTM(32, stateful=True))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((batch_size * 10, timesteps, data_dim))
y_train = np.random.random((batch_size * 10, num_classes))

# Generate dummy validation data
x_val = np.random.random((batch_size * 3, timesteps, data_dim))
y_val = np.random.random((batch_size * 3, num_classes))

model.fit(x_train, y_train, batch_size=batch_size, epochs=5, 
          shuffle=False, validation_data=(x_val, y_val))

score = model.evaluate(x_val, y_val, batch_size=batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 320 samples, validate on 96 samples
Epoch 1/5
320/320 [==============================] - 2s 7ms/step - loss: 11.4301 - acc: 0.1313 - val_loss: 11.4971 - val_acc: 0.0521
Epoch 2/5
320/320 [==============================] - 0s 242us/step - loss: 11.4246 - acc: 0.1219 - val_loss: 11.4976 - val_acc: 0.0521
Epoch 3/5
320/320 [==============================] - 0s 250us/step - loss: 11.4233 - acc: 0.1187 - val_loss: 11.4980 - val_acc: 0.0521
Epoch 4/5
320/320 [==============================] - 0s 236us/step - loss: 11.4223 - acc: 0.1156 - val_loss: 11.4985 - val_acc: 0.0521
Epoch 5/5
96/96 [==============================] - 0s 129us/step
Test loss: 11.5005394618
Test accuracy: 0.0625
